In [ ]:
# %% [markdown]
# # Microglia Phenotyping Tool - Quick Start
# This notebook demonstrates basic usage of the microglia analysis pipeline.

# %% [markdown]
# ## 1. Import the analysis module

# %%
from cell_analysis import main
import pandas as pd
from IPython.display import Image, display
import os

# %% [markdown]
# ## 2. Set up paths
# Specify your input image and output directory:

# %%
# Path to your 4-channel TIFF image
image_path = "path/to/your/image.tif"  # Update this!

# Output directory for results
output_dir = "results/sample1"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# %% [markdown]
# ## 3. Run the analysis
# **Channel assignments:**
# - Channel 0: DAPI (nuclei)
# - Channel 1: QKI (protein marker)
# - Channel 2: IBA1 (microglia)
# - Channel 3: CD3 (T cells)

# %%
results = main(
    image_path=image_path,
    output_dir=output_dir,
    qki_channel=1,
    iba1_channel=2,
    dapi_channel=0,
    tcell_channel=3,
    max_workers=8  # Adjust based on your CPU cores
)

# %% [markdown]
# ## 4. View summary statistics

# %%
print("=== Analysis Summary ===")
print(f"IBA1+ microglia: {results['stats']['iba1_count']}")
print(f"T cells: {results['stats']['tcell_count']}")
print(f"DAPI+ nuclei: {results['stats']['nuclei_count']}")
print(f"\nMorphology distribution:")
for morph_type, count in results['stats']['morphology_counts'].items():
    if morph_type != 'artifact':
        print(f"  {morph_type.capitalize()}: {count}")

# %% [markdown]
# ## 5. Display visualizations

# %%
# Multi-channel composite with cell boundaries
print("Composite Image:")
display(Image(filename=os.path.join(output_dir, "composite.png"), width=800))

# %%
# Morphology classification
print("Morphology Classification:")
display(Image(filename=os.path.join(output_dir, "morphology.png"), width=800))

# %%
# Morphology distribution pie chart
print("Morphology Distribution:")
display(Image(filename=os.path.join(output_dir, "morphology_pie_chart.png"), width=600))

# %% [markdown]
# ## 6. Explore the data

# %%
# Load comprehensive cell data
cells_df = pd.read_csv(os.path.join(output_dir, "all_cells_comprehensive.csv"))

print(f"Total cells analyzed: {len(cells_df)}")
print(f"\nFirst 5 cells:")
display(cells_df.head())

# %%
# View IBA1+ cells only
iba1_cells = cells_df[cells_df['cell_type'] == 'iba1']
print(f"IBA1+ cells: {len(iba1_cells)}")
display(iba1_cells[['cell_id', 'morphology_class', 'area', 'nuclei_count', 'avg_nuclear_qki']].head(10))

# %%
# View T cells only
tcells = cells_df[cells_df['cell_type'] == 'tcell']
print(f"T cells: {len(tcells)}")
display(tcells[['cell_id', 'centroid_x', 'centroid_y', 'area']].head(10))

# %% [markdown]
# ## 7. View spatial analysis

# %%
# Load spatial analysis results
spatial_df = pd.read_csv(os.path.join(output_dir, "spatial_analysis_summary_detailed.csv"))

print("Spatial Analysis Summary:")
print(f"Average microglia-to-T cell distance: {spatial_df['avg_macro_to_tcell_distance_um'].values[0]:.2f} μm")
display(spatial_df.T)

# %%
# Display density heatmap
print("Combined Density Heatmap (Red=IBA1, Green=T cells):")
display(Image(filename=os.path.join(output_dir, "combined_density_heatmap.png"), width=800))

# %% [markdown]
# ## 8. Output files generated
# 
# The analysis creates the following files in your output directory:
# 
# **CSV Data:**
# - `all_cells_comprehensive.csv` - All cells with full metrics
# - `iba1_cells.csv` - IBA1+ cells only
# - `tcells.csv` - T cells only
# - `stats.csv` - Overall statistics
# - `macrophage_centered_analysis_detailed.csv` - Spatial analysis from microglia perspective
# - `tcell_centered_analysis_detailed.csv` - Spatial analysis from T cell perspective
# 
# **Visualizations:**
# - `composite.png` - Multi-channel overlay
# - `morphology.png` - Morphology classification map
# - `morphology_pie_chart.png` - Distribution chart
# - `combined_density_heatmap.png` - Spatial density
# - And more...

# %%
# List all generated files
print("Generated files:")
for file in sorted(os.listdir(output_dir)):
    print(f"  - {file}")

# %% [markdown]
# ## Done!
# 
# You can now explore the CSV files and visualizations in your output directory.
# 
# For customization options, see the README.md or check the `main()` function parameters in `cell_analysis.py`.